Create problem

In [1]:
	
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic

In [2]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [3]:
problem = create_problem(10, density=0.15, noise_level=10, negative_values=False)

In [4]:
problem

array([[    0.,    inf, 10573.,    inf,  8430.,    inf,    inf,   831.,
         1977.,    inf],
       [   inf,     0.,    inf,    inf,    inf,  2434.,    inf,    inf,
           inf,  6771.],
       [   inf,    inf,     0.,    inf,    inf,    inf,    inf,    inf,
           inf,  2208.],
       [   inf,    inf,  8563.,     0.,  7768.,    inf,    inf,    inf,
           inf,    inf],
       [ 7330.,    inf,    inf,  8367.,     0.,    inf,    inf,    inf,
           inf,    inf],
       [   inf,    inf,    inf,    inf,    inf,     0.,    inf,    inf,
           inf,    inf],
       [   inf,    inf,    inf,    inf,    inf,    inf,     0.,  5247.,
           inf,    inf],
       [   inf,    inf,  5287.,    inf,    inf,    inf,    inf,     0.,
           inf,    inf],
       [   inf,    inf,    inf, 10443.,  8363.,    inf,  5258.,    inf,
            0.,    inf],
       [   inf,    inf,    inf,    inf,    inf,    inf,  7850.,  7752.,
           inf,     0.]])

In [5]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [6]:
for s, d in combinations(range(problem.shape[0]), 2):
    try:
        path = nx.shortest_path(G, s, d, weight='weight')
        cost = cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        path = None
        cost = np.inf
    ic(s, d, path, cost)
None

ic| s: 0, d: 1, path: None, cost: inf
ic| s: 0, d: 2, path: [0, 7, 2], cost: 6118.0
ic| s: 0, d: 3, path: [0, 8, 3], cost: 12420.0
ic| s: 0, d: 4, path: [0, 4], cost: 8430.0
ic| s: 0, d: 5, path: None, cost: inf
ic| s: 0, d: 6, path: [0, 8, 6], cost: 7235.0
ic| s: 0, d: 7, path: [0, 7], cost: 831.0
ic| s: 0, d: 8, path: [0, 8], cost: 1977.0
ic| s: 0, d: 9, path: [0, 7, 2, 9], cost: 8326.0
ic| s: 1, d: 2, path: [1, 9, 7, 2], cost: 19810.0
ic| s: 1, d: 3, path: None, cost: inf
ic| s: 1, d: 4, path: None, cost: inf
ic| s: 1, d: 5, path: [1, 5], cost: 2434.0
ic| s: 1, d: 6, path: [1, 9, 6], cost: 14621.0
ic| s: 1, d: 7, path: [1, 9, 7], cost: 14523.0
ic| s: 1, d: 8, path: None, cost: inf
ic| s: 1, d: 9, path: [1, 9], cost: 6771.0
ic| s: 2, d: 3, path: None, cost: inf
ic| s: 2, d: 4, path: None, cost: inf
ic| s: 2, d: 5, path: None, cost: inf
ic| s: 2, d: 6, path: [2, 9, 6], cost: 10058.0
ic| s: 2, d: 7, path: [2, 9, 7], cost: 9960.0
ic| s: 2, d: 8, path: None, cost: inf
ic| s: 2, d: 9, pat

parameter

size = [10, 20, 50, 100, 200, 500, 1000]
density = [0.2, 0.5, 0.8, 1]
noise_level = [0.0, 0.1, 0.5, 0.8]
negative_values = [F, T]


In [7]:
#start code

# > 0 Dijkstra
def solve_instance_dijkstra(
        size: int,
        density: float,
        noise_level: float,
        negative_values: bool,
        seed: int = 42,  
        s: int = int,
        d: int = int  
) -> float:
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=seed
    )

    if(problem < 0).any():
        print("Error: Negative values in the problem matrix.")

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)
    try:
        path = nx.shortest_path(G, source = s, target = d, weight='weight')
        cost = nx.path_weight(G, path, weight='weight')
    except nx.NetworkXNoPath:
        path = None
        cost = np.inf
    return problem, path, cost


In [14]:
# <0 Bellman-Ford

def solve_instance_bellman_ford(
        size: int,
        density: float,
        noise_level: float,
        negative_values: bool,
        seed: int = 42,
        s: int = 0,
        d: int = 1,
):
    
    problem = create_problem(
        size,
        density=density,
        noise_level=noise_level,
        negative_values=negative_values,
        seed=seed
    )

    masked = np.ma.masked_array(problem, mask=np.isinf(problem))
    G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

    
    try:
        path = nx.bellman_ford_path(G, source=s, target=d, weight="weight")
        cost = nx.path_weight(G, path, weight="weight")

    except nx.NetworkXNoPath:
        path = None
        cost = np.inf

    except nx.NetworkXUnbounded:
        path = None
        cost = -np.inf

    return problem, path, cost

if cost == np.inf:
    print(f"No path from {s} to {d}")
elif cost == -np.inf:
    print(f"There is a negative cycle reachable from {s} that can affect {d}; shortest path is undefined.")
elif cost <= 0:
    print(f"Path exists but cost = {cost} (not positive, so we discard it).")
else:
    print(f"Shortest path from {s} to {d}: {path} with cost {cost}")


There is a negative cycle reachable from 0 that can affect 49; shortest path is undefined.


In [15]:
#test bellman ford
size = 50
density = 0.5
noise_level = 10
negative_values = True   # allow negative values
seed = 42
s = 0
d = size - 1

problem, path, cost = solve_instance_bellman_ford(
    size=size,
    density=density,
    noise_level=noise_level,
    negative_values=negative_values,
    seed=seed,
    s=s,
    d=d,
)


print("Problem Matrix:\n", problem)
print(f"Shortest path from {s} to {d}: {path} with cost: {cost}")


Problem Matrix:
 [[    0.    inf -3816. ...    inf    inf    inf]
 [   inf     0.    inf ...  8052. -5285.    inf]
 [   inf    inf     0. ... -1346. -1589. -1455.]
 ...
 [ 3656.    inf  3234. ...     0.  6935.    inf]
 [-8781.    inf    inf ...    inf     0.    inf]
 [ 1705.   772.    inf ...    inf    inf     0.]]
Shortest path from 0 to 49: None with cost: -inf


In [ ]:
for size in [10, 50, 100]:
    for density in [0.2, 0.5]:
        problem, path, cost = run_instance(
            size=size,
            density=density,
            noise_level=0.5,
            negative_values=False,
        )

print("Problem Matrix:\n", problem)
print(f"Shortest path from {s} to {d}: {path} with cost: {cost}")


Problem Matrix:
 [[   0.   inf  999. ...   inf   inf   inf]
 [  inf    0.   inf ...  998.  885.   inf]
 [  inf   inf    0. ...  609. 1115.  240.]
 ...
 [ 838.   inf  724. ...    0.  956.   inf]
 [ 558.   inf   inf ...   inf    0.   inf]
 [1111. 1036.   inf ...   inf   inf    0.]]
Shortest path from 0 to 49: [0, 39, 49] with cost: 1042.0
